In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import datetime
from time import sleep

In [2]:
chromedriver_path = "C:/Users/elisa/Projet Python/chromedriver_win32/chromedriver.exe"
#chromedriver_path = "C:/Users/bombi/Downloads/chromedriver_win32/chromedriver.exe"
#chromedriver_path = 'C:/Users/José/Documents/ENSAE Paris/2A/Python Pour le Data Scientist/chromedriver_win32/chromedriver.exe'

In [3]:
driver = webdriver.Chrome(chromedriver_path)

In [4]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('span',class_='codeshares-airline-names',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('div',class_='section stops')

    for i in stops:
        for j in i.find_all('span',class_='stops-text'):
               stops_list.append(j.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='Flights-Results-FlightPriceSection right-alignment sleek')

    for i in price:
        for j in i.find_all('span', class_='price-text'):
            prices.append(j.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='section duration allow-multi-modal-icons')
    for i in duration:
        for j in i.find_all('div',class_='top'):
            duration_list.append(j.text)
    return duration_list

In [5]:
def remanier(compagnie):
    liste = []
    for i in range(len(compagnie)):
        if  ',' in compagnie[i] : 
            j= 0
            while compagnie[i][j]!=',' :
                j+=1
            liste.append(compagnie[i][:j])
            liste.append(compagnie[i][j+1:])
        else :
            liste.append(compagnie[i])
            liste.append(compagnie[i])
    return liste

In [6]:
def remanier_2(prix):
    liste = []
    for i in range(len(prix)):
        liste.append(prix[i])
        liste.append(prix[i])
    return liste

In [7]:
depart = ["PAR"]
Destinations = ["MAD", "SVO", "ATH", "OSL", "LON"]
date_aller = ["07/01/2022"]
date_retour = ["07/08/2022"]
date_a = pd.to_datetime(date_aller)
date_r = pd.to_datetime(date_retour)
result = pd.DataFrame([])

for k in range(13): 
    new_date_a = date_a + datetime.timedelta(days = 7*k)
    new_date_r = date_r + datetime.timedelta(days = 7*k)
    date_time_a = new_date_a.astype(str)
    date_time_r = new_date_r.astype(str)
    destination = ["LON"]
    date_aller = [date_time_a]        
    date_retour = [date_time_r]
    url = f"https://www.kayak.fr/flights/{depart[0]}-{destination[0]}/{date_aller[0]}/{date_retour[0]}"
    driver.get(url)
    sleep(15)
    
    try:
        show_more_button = driver.find_element_by_xpath('//a[@class = "moreButton"]')
    except:
        input("Please solve the captcha then enter anything here to resume scraping.")

    while True:
        try:
            show_more_button.click()
            driver.implicitly_wait(10)
        except:
            break


    soup = BeautifulSoup(driver.page_source, 'html.parser')
    compagnie = remanier(get_airlines(soup))
    total_escales = get_total_stops(soup)
    prix = remanier_2(get_price(soup))
    duree = get_duration(soup)
    df = pd.DataFrame({'Compagnie':compagnie, "Total d'escales":total_escales , 'Prix': prix, 'Duree': duree}, 
                        columns = ['Compagnie', "Total d'escales", 'Prix', 'Duree'])
    df["Destination"] = "Moscou"
    df["Distance"] = 342.76
    df["Date_aller"] = 20220701
    df["Date_retour"] = 20220708
    df["Date_aller"] = pd.to_datetime(df["Date_aller"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    df["Date_retour"] = pd.to_datetime(df["Date_retour"], format = '%Y%m%d') + datetime.timedelta(days = 7*k)
    result = pd.concat([result, df], axis = 0)

In [11]:
result['Destination']='Londres'

In [12]:
result

,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
0,easyJet,\ndirect,\n59 €,\n1h 20min,Londres,342.76,2022-07-01,2022-07-08
1,easyJet,\ndirect,\n59 €,\n1h 20min,Londres,342.76,2022-07-01,2022-07-08
2,easyJet,\ndirect,\n62 €,\n1h 15min,Londres,342.76,2022-07-01,2022-07-08
3,easyJet,\ndirect,\n62 €,\n1h 20min,Londres,342.76,2022-07-01,2022-07-08
4,easyJet,\ndirect,\n64 €,\n1h 05min,Londres,342.76,2022-07-01,2022-07-08
...,...,...,...,...,...,...,...,...
2327,Iberia,\ndirect,\n117 €,\n1h 20min,Londres,342.76,2022-09-23,2022-09-30
2328,easyJet,\ndirect,\n153 €,\n1h 10min,Londres,342.76,2022-09-23,2022-09-30
2329,Air France,\ndirect,\n153 €,\n1h 25min,Londres,342.76,2022-09-23,2022-09-30
2330,British Airways,\ndirect,\n159 €,\n1h 20min,Londres,342.76,2022-09-23,2022-09-30


In [13]:
result.to_excel('londres_2.xlsx', index = False) 

In [28]:
fichiers = ["C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/oslo.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/oslo_2.xlsx","C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/moscou.xlsx","C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/madrid.xlsx","C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/madrid_2.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/londres.xlsx","C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/londres_1.xlsx", "C:/Users/bombi/OneDrive/Documents/Fichiers_Scrappes/athenes.xlsx"]
fichier_combine= pd.DataFrame()

for fichier in fichiers:
    df=pd.read_excel(fichier)
    fichier_combine= fichier_combine.append(df, ignore_index= True)

In [30]:
fichier_combine

,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
0,Air France,\ndirect,\n204 €,\n2h 20min,Oslo,1344.00,2022-04-01,2022-04-08
1,Air France,\ndirect,\n204 €,\n2h 30min,Oslo,1344.00,2022-04-01,2022-04-08
2,Flyr AS,\ndirect,\n195 €,\n2h 25min,Oslo,1344.00,2022-04-01,2022-04-08
3,Flyr AS,\ndirect,\n195 €,\n2h 25min,Oslo,1344.00,2022-04-01,2022-04-08
4,Norwegian,\ndirect,\n186 €,\n2h 20min,Oslo,1344.00,2022-04-01,2022-04-08
...,...,...,...,...,...,...,...,...
61343,Turkish Airlines,\n1 escale,\n294 €,\n7h 20min,Athenes,2095.52,2022-06-10,2022-06-17
61344,Sky Express,\ndirect,\n1 874 €,\n3h 00min,Athenes,2095.52,2022-06-10,2022-06-17
61345,SWISS,\n1 escale,\n1 874 €,\n5h 30min,Athenes,2095.52,2022-06-10,2022-06-17
61346,Sky Express,\ndirect,\n1 879 €,\n3h 00min,Athenes,2095.52,2022-06-10,2022-06-17


In [27]:
#fichier_combine.to_excel('fich-comb.xlsx')

In [31]:
df_final = fichier_combine

In [32]:
df_final

,Compagnie,Total d'escales,Prix,Duree,Destination,Distance,Date_aller,Date_retour
0,Air France,\ndirect,\n204 €,\n2h 20min,Oslo,1344.00,2022-04-01,2022-04-08
1,Air France,\ndirect,\n204 €,\n2h 30min,Oslo,1344.00,2022-04-01,2022-04-08
2,Flyr AS,\ndirect,\n195 €,\n2h 25min,Oslo,1344.00,2022-04-01,2022-04-08
3,Flyr AS,\ndirect,\n195 €,\n2h 25min,Oslo,1344.00,2022-04-01,2022-04-08
4,Norwegian,\ndirect,\n186 €,\n2h 20min,Oslo,1344.00,2022-04-01,2022-04-08
...,...,...,...,...,...,...,...,...
61343,Turkish Airlines,\n1 escale,\n294 €,\n7h 20min,Athenes,2095.52,2022-06-10,2022-06-17
61344,Sky Express,\ndirect,\n1 874 €,\n3h 00min,Athenes,2095.52,2022-06-10,2022-06-17
61345,SWISS,\n1 escale,\n1 874 €,\n5h 30min,Athenes,2095.52,2022-06-10,2022-06-17
61346,Sky Express,\ndirect,\n1 879 €,\n3h 00min,Athenes,2095.52,2022-06-10,2022-06-17
